# Bayessche logistische Regression – UX Vorhersagemodell

Autor: Luca Honegger, lh@sinnhaft.ch

Quelle: Dies ist eine Python-Portierung der UXtoolbox in R von Mohsen Rafiei, https://github.com/mohsen-rafiei/UXtoolbox


## 🧠 Was macht diese Funktion?

Die Funktion `bayesian_logistic_regression()` schätzt, **mit welcher Wahrscheinlichkeit ein binäres Nutzerergebnis eintritt** – z. B. ob Nutzer\:innen eine Aufgabe erfolgreich abgeschlossen haben oder nicht.

Im Unterschied zu klassischen Regressionsverfahren gibt sie nicht nur einen festen Wert aus, sondern **modelliert die Unsicherheit explizit** – durch **Posterior-Verteilungen**, **glaubwürdige Intervalle** und **vorhergesagte Wahrscheinlichkeiten**.

Sie basiert auf einem **Bayesschen Regressionsansatz**, der besonders bei kleinen Stichproben und in der UX-Forschung (A/B-Tests, Umfragen, Conversion-Analyse) nützlich ist.

### 🔍 Was wird berechnet?

Die Funktion führt eine **Bayessche logistische Regression** durch:

* Die **Zielvariable** (z. B. `task_success`) ist binär (0 = Misserfolg, 1 = Erfolg)
* Die **Einflussfaktoren** (Prädiktoren) können kontinuierlich oder kategorisch sein (z. B. `experience`, `age`)
* Die Unsicherheit der Regressionsparameter wird durch **Posterior-Samples** abgebildet
* Zusätzlich werden **95 %-Credible Intervals** und **vorhergesagte Erfolgswahrscheinlichkeiten** berechnet

Formelhaft entspricht das dem Modell:

$$
\text{logit}(p) = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n
$$

Mit Bayes’schem Update ergibt sich für jede \$\beta\$ eine Wahrscheinlichkeitsverteilung statt eines festen Werts.

### 🧪 Beispiel aus dem UX-Alltag

Stell dir vor, du möchtest herausfinden, wie **Erfahrung** und **Alter** die Erfolgsrate in einem Usability-Test beeinflussen. Du hast folgende Daten von 5 Testpersonen:

| task\_success | experience | age |
| ------------- | ---------- | --- |
| 1             | 1          | 22  |
| 0             | 2          | 30  |
| 1             | 3          | 25  |
| 1             | 4          | 40  |
| 0             | 5          | 35  |

Mit folgendem Aufruf kannst du das Modell schätzen:

```python
bayesian_logistic_regression(data, formula="task_success ~ experience + age")
```

Die Funktion liefert dir:

* Eine **Modellzusammenfassung** mit Mittelwerten und Streuungen der Koeffizienten
* Die **glaubwürdigen Intervalle** für alle Parameter
* Die **vorhergesagten Wahrscheinlichkeiten**, ob eine Person mit bestimmten Merkmalen erfolgreich wäre


### 🎯 Warum ist das wichtig?

Bayessche Modelle sind in der UX-Forschung besonders nützlich, wenn:

* **kleine Stichproben** vorliegen (z. B. bei Nutzertests)
* **Unsicherheiten explizit** berücksichtigt werden sollen
* **Vorwissen (Priors)** einbezogen werden kann – z. B. aus früheren Studien

Mit dieser Funktion kannst du deine UX-Daten **probabilistisch analysieren**, statt dich nur auf starre Signifikanztests zu verlassen – und so **robustere Entscheidungen treffen**.


### 📝 Hinweise
- Das Modell verwendet standardmässig Normalverteilungen mit mu=0, sigma=1 als schwach informative Priors.
- Die Formel-Syntax ist einfach gehalten (kein Interaktions-Term oder Categorical).

## ⚙️ `bayesian_logistic_regression`

In [66]:
import pymc as pm
import arviz as az
import pandas as pd
import numpy as np

def bayesian_logistic_regression(data, formula_terms, target_column, n_samples=1000, prior_std=1.0):
    """
    Führt eine bayessche logistische Regression durch mit Normalverteilungen als Prioren.

    Die Funktion schätzt die Erfolgswahrscheinlichkeit eines binären Outcomes
    (z. B. Task-Erfolg) in Abhängigkeit von mehreren Prädiktoren (z. B. Erfahrung, Alter).
    Als Ergebnis liefert sie Posteriorverteilungen, 95 % Credible Intervals
    und vorhergesagte Wahrscheinlichkeiten.

    Parameter:
    - data: Pandas DataFrame mit den UX-Daten
    - formula_terms: Liste der erklärenden Variablen (z. B. ["experience", "age"])
    - target_column: Name der Zielvariablen (z. B. "task_success")
    - n_samples: Anzahl der MCMC-Samples pro Kette (Standard: 1000)
    - prior_std: Standardabweichung der Normalverteilung für die Priors (Standard: 1.0)

    Rückgabe:
    - Dictionary mit:
        - 'model': Das PyMC-Modellobjekt
        - 'trace': Die gezogenen Posterior-Samples
        - 'summary': Statistische Zusammenfassung mit Credible Intervals
        - 'predicted_probabilities': Mittlere Erfolgswahrscheinlichkeiten je Datenpunkt
    """
    
    # 1. Vorbereitung der Daten
    X = data[formula_terms].values
    X = (X - X.mean(axis=0)) / X.std(axis=0)  # Standardisierung
    y = data[target_column].values

    # 2. Bayesianisches Modell definieren
    with pm.Model() as model:
        # Priors für Regressionskoeffizienten
        intercept = pm.Normal("intercept", mu=0, sigma=prior_std)
        coefs = pm.Normal("coefs", mu=0, sigma=prior_std, shape=X.shape[1])

        # Linearkombination und logistische Verknüpfung
        logits = intercept + pm.math.dot(X, coefs)
        p = pm.Deterministic("p", pm.math.sigmoid(logits))

        # Likelihood (Beobachtete Daten)
        likelihood = pm.Bernoulli("likelihood", p=p, observed=y)

        # 3. Posterior-Samples mit NUTS ziehen
        trace = pm.sample(draws=n_samples, chains=4, target_accept=0.9)

    # 4. Zusammenfassung
    summary = az.summary(trace, hdi_prob=0.95)
    pred_probs = trace.posterior["p"].mean(dim=("chain", "draw")).values

    return {
        "model": model,
        "trace": trace,
        "summary": summary,
        "predicted_probabilities": pred_probs
    }

## 🚀 Anwendungsbeispiel

In [80]:
# Beispiel-Datensatz
data = pd.DataFrame({
    "task_success": [1, 0, 1, 1, 0, 1, 0],
    "experience": [5, 2, 4, 4, 2, 4, 1],
    "age": [22, 30, 55, 40, 35, 25, 57]
})

result = bayesian_logistic_regression(
    data=data,
    formula_terms=["experience", "age"],
    target_column="task_success"
)

# Ausgabe
print(result["summary"])

 Progress                   Draws   Divergences   Step size   Grad evals   Sampling Speed    Elapsed   Remaining  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  ━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.71        7            3658.11 draws/s   0:00:00   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.83        7            3689.36 draws/s   0:00:00   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.99        7            3480.93 draws/s   0:00:00   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.81        3            3344.82 draws/s   0:00:00   0:00:00

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


            mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
coefs[0]   1.491  0.741     0.105      2.956      0.012    0.012    3953.0   
coefs[1]  -0.038  0.726    -1.461      1.346      0.013    0.011    3326.0   
intercept  0.204  0.703    -1.200      1.570      0.012    0.011    3733.0   
p[0]       0.847  0.155     0.507      1.000      0.002    0.003    3995.0   
p[1]       0.300  0.187     0.019      0.671      0.003    0.003    3595.0   
p[2]       0.693  0.224     0.258      0.998      0.004    0.003    3418.0   
p[3]       0.728  0.154     0.427      0.976      0.003    0.002    3693.0   
p[4]       0.292  0.170     0.025      0.624      0.003    0.003    3774.0   
p[5]       0.723  0.178     0.373      0.988      0.003    0.002    3753.0   
p[6]       0.176  0.188     0.000      0.602      0.003    0.003    4271.0   

           ess_tail  r_hat  
coefs[0]     3284.0    1.0  
coefs[1]     2730.0    1.0  
intercept    3036.0    1.0  
p[0]         3005.0    1.

### 📖 Erklärung und Interpretation

#### Überblick: Was zeigt das Modell?

Das Modell ist eine **Bayessche logistische Regression** mit zwei Prädiktoren (`coefs[0]` und `coefs[1]`) und einem Intercept. Die Zielvariable ist binär (z. B. „Task erfolgreich abgeschlossen: ja/nein“).

Die Schätzung basiert auf 4 MCMC-Ketten mit je 2000 Samples (insgesamt 8000 Samples), ohne Divergenzen und mit sehr guter Konvergenz (`r_hat = 1.0` bei allen Parametern).


#### Modellparameter – Interpretation

| Parameter   | Mittelwert (mean) | 95 %-Credible Interval | Interpretation                                  |
| ----------- | ----------------- | ---------------------- | ----------------------------------------------- |
| `intercept` | 0.204             | \[-1.200, 1.570]       | Grundwahrscheinlichkeit (logit), wenn x=0       |
| `coefs[0]` (Erfahrung)  | **1.491**         | \[**0.105**, 2.956]    | Erfahrung: Positiver Einfluss, **statistisch glaubwürdig** |
| `coefs[1]` (Alter)  | -0.038            | \[-1.461, 1.346]       | Alter: Kein klarer Effekt erkennbar                    |

#### Deutung:

* `coefs[0]` (Erfahrung): Zeigt **einen glaubwürdigen positiven Einfluss** auf den Erfolg, weil das gesamte 95 %-Intervall über 0 liegt.
* `coefs[1]` (Alter): Der Effekt ist unsicher; das Intervall umfasst sowohl negative als auch positive Werte → keine Aussage möglich.
* `intercept`: Dient nur als Basislinie – Interpretation ist weniger entscheidend.


#### Vorhersagen (individuelle Erfolgswahrscheinlichkeiten)

| Beobachtung | p\[i] (mean) | 95 %-Credible Interval | Interpretation                         |
| ----------- | ------------ | ---------------------- | -------------------------------------- |
| p\[0]       | 0.847        | \[0.507, 1.000]        | Sehr hohe Erfolgswahrscheinlichkeit    |
| p\[1]       | 0.300        | \[0.019, 0.671]        | Eher geringe Erfolgswahrscheinlichkeit |
| p\[2]       | 0.693        | \[0.258, 0.998]        | Gute Erfolgschance                     |
| p\[3]       | 0.728        | \[0.427, 0.976]        | Gute Erfolgschance                     |
| p\[4]       | 0.292        | \[0.025, 0.624]        | Eher geringe Chance                    |
| p\[5]       | 0.723        | \[0.373, 0.988]        | Gute Chance                            |
| p\[6]       | 0.176        | \[0.000, 0.602]        | Sehr geringe Erfolgswahrscheinlichkeit |

✅ Die Werte unterscheiden sich deutlich – das Modell **erkennt individuelle Unterschiede**.


#### Qualität der Schätzung

* ✅ **Alle `r_hat = 1.0`** → sehr gute Konvergenz
* ✅ **Keine Divergenzen** → Sampling stabil
* ✅ **ESS > 2700** bei allen Parametern → hohe Zuverlässigkeit
* ✅ **Niedrige `mcse`-Werte** → präzise Posterior-Schätzungen


#### Business-Interpretation

#### Möglicher Kontext:

Ein UX- oder Produkttest untersucht, ob z. B. **Erfahrung (`coefs[0]`)** und **Alter (`coefs[1]`)** den Erfolg in einem Interface-Test beeinflussen.

#### Ergebnisse & Empfehlungen:

| Erkenntnis                                       | Auswirkung                                                                                                                                             |
| ------------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------ |
| ✅ Prädiktor 1 "Erfahrung" hat glaubwürdigen positiven Effekt | Nutzer mit höherer Ausprägung in `coefs[0]` (z. B. Erfahrung, Training, Onboarding-Stufe) haben **signifikant höhere Erfolgsraten**                    |
| ❌ Prädiktor 2 "Alter" liefert keine klare Aussage        | Der zweite Prädiktor (z. B. Alter, kognitive Last?) sollte **nicht zur Entscheidungsgrundlage** gemacht werden                                         |
| 📊 Klare Unterschiede in Zielgruppen             | Einige Nutzergruppen (z. B. `p[0]`, `p[2]`, `p[3]`) haben **sehr hohe Erfolgschancen**, andere (z. B. `p[1]`, `p[4]`, `p[6]`) **deutlich schlechtere** |



#### 📌 Fazit

| Massnahme                                                   | Begründung                                                        |
| ---------------------------------------------------------- | ----------------------------------------------------------------- |
| 🎯 Fokus auf 'Erfahrung' `coefs[0]` erhöhen                            | Dieser Prädiktor ist **relevant und wirksam**                     |
| 🔁 'Alter' `coefs[1]` prüfen oder durch relevanteres Mass ersetzen | Keine stabile Aussage erkennbar                                   |
| 📈 UX-Massnahmen für `p[6]`, `p[4]`, `p[1]` entwickeln      | Diese Nutzer\:innen sind besonders gefährdet, zu scheitern        |
| 🔬 Weitere qualitative Interviews           | Um herauszufinden, **warum bestimmte Gruppen erfolgreicher sind** |
